In [ ]:
import os
import glob
from pprint import pprint
import json
import pickle
import re
from collections import OrderedDict
from Bio import SeqIO

In [ ]:
from rdkit import Chem as chem
from rdkit.Chem import AllChem, Draw
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import imp
helper = imp.load_source('helper', './pks/helper.py')
domain = imp.load_source('domain', './pks/domain.py')
pks = imp.load_source('pks', './pks/pks.py')

In [ ]:
file_path = './mibig'
file_names = glob.glob(os.path.join(file_path, '*.json'))

# MiBiG Breakdown

In [ ]:
# Erythromycin
test_file = './mibig/BGC0000055.json'
with open(test_file) as json_file:
    test_data = json.load(json_file)
#pprint(test_data.keys())
#pprint(test_data['general_params'].keys())
#pprint(test_data['general_params'])
pprint(test_data['general_params']['loci']['nucl_acc'][0]['Accession'])
pprint(test_data['general_params']['Polyketide'])
#pprint([compound['compound'] for compound in test_data['general_params']['compounds']])

In [ ]:
# See what kind of PKS subtype labels there are
labels = []
for file_name in file_names:
    with open(file_name) as json_file:
        json_data = json.load(json_file)
    try:
        labels.extend(json_data['general_params']['Polyketide']['pks_subclass'])
    except KeyError:
        pass
print(set(labels))

In [ ]:
# This gets all the modular type I PKSs contained in MiBiG (at least the examples that are so labeled)
t1pks = []
for file_name in file_names:
    with open(file_name) as json_file:
        json_data = json.load(json_file)
    try:        
        if len(set(['Modular type I', 'Modular Type I', 'Type I']).intersection(set(json_data['general_params']['Polyketide']['pks_subclass']))) > 0:
            accession = json_data['general_params']['loci']['nucl_acc'][0]['Accession']
            t1pks.append((file_name.split('/')[-1].split('.')[0], accession))
    except KeyError:
        pass
#print(len(t1pks))
#print([entry[1] for entry in t1pks])
# with open('./mibig/antismash/t1pks_list.txt', 'w') as f:
#     for entry in t1pks:
#         f.write(entry[0].split('/')[-1].split('.')[0] + '\n')

In [ ]:
# Number of clusters found
print('%d potential type I modular PKS clusters found!' %(len(t1pks)))
# Iterate over list of type I modular PKSs
for i in range(len(t1pks)):
    entry = t1pks[i]
    # This prints the product compounds for the clusters
    with open(os.path.join(file_path, entry[0] + '.json')) as json_file:
        mibig_data = json.load(json_file)
        pprint([compound['compound'] for compound in mibig_data['general_params']['compounds']])

In [ ]:
# Get keys for genes
keys = []
for i in range(len(t1pks)):
    entry = t1pks[i]
    # This prints the product compounds for the clusters
    with open(os.path.join(file_path, entry[0] + '.json')) as json_file:
        mibig_data = json.load(json_file)
        # Iterate over all subunits getting domains
        keys.extend(list(mibig_data['general_params']['Polyketide'].keys()))
print(set(keys))

In [ ]:
# Get set of catalytic domains found across all type I modular PKSs
domains = []
lin_cyc = []
for i in range(len(t1pks)):
    entry = t1pks[i]
    # This prints the product compounds for the clusters
    with open(os.path.join(file_path, entry[0] + '.json')) as json_file:
        mibig_data = json.load(json_file)
        # Iterate over all subunits getting domains
        try:
            pks_genes = mibig_data['general_params']['Polyketide']['mod_pks_genes']
            lin_cyc.append(mibig_data['general_params']['Polyketide']['lin_cycl_pk'])
        except KeyError:
            continue
        for subunit in pks_genes:
            subunit_name = re.sub(r'\s+', '', subunit['mod_pks_gene'])
            subunit_modules = subunit['pks_module']
            for module in subunit_modules:
                domains.extend(module['pks_domains'])
print(set(domains))
print(set(lin_cyc))

In [ ]:
# antiSMASH output from Tyler
antismash_file_path = './mibig/antismash/split_files'

# Proccessing functions

In [ ]:
def process_subunit_modules(sec_met): 
    '''This function takes as input the the list recorded by feature.qualifiers['sec_met'] for a module in a PKS
       cluster. This assumes that feature.type=='CDS' and that feature.qualifiers has the key 'sec_met'.
       The function returns a dict corresponding to the modules in the subunit, indexed starting from zero within
       the subunit. If the first entry of 'sec_met' is not 'Type: t1pks' then nothing is returned.
    '''
    # Initialize dict for the subunit
    # keys: module number
    # values: OrderedDict of domains in module
    #         within OrderedDict, key is domain name and value is a length 2 list where the
    #         first element is a dictionary {start:, stop:} and the second element is specificity dictionary 
    subunit = {}
    
    # This is for the current module (function processes subunit which may have more than one module)
    module_index = 0  # key for module
    module_domains = [] # list of domains in module
    old_module_domains = [] # pre-initialize in case subunit starts with a domain
                            # that is expected to end the module
    
    # This is how domains appear in sec_met:
    # ['PKS_AT', 'PKS_KS', 'PKS_KR', 'PKS_DH', 'PKS_ER', 'ACP', 'Thioesterase']
    # Iterate over the entries in sec_met, and add them to the module_domains list 
    for entry in sec_met:    
        # Split entry into a list
        entrysplit = [item.strip() for item in entry.split(';') if item != '']
        # Split part of entry that is expected to describe catalytic domain
        domainsplit = entrysplit[0].split()
        # This is just different ways of processing the name of the domain depending
        # on how the name of the domain is formatted
        if ' '.join(domainsplit[:2]) == 'NRPS/PKS Domain:' and len(domainsplit) > 2:
            # Note that we want to make sure that there is a leading 'PKS_' before we do our trimming
            if domainsplit[2].split('_')[0] == 'PKS':
                if domainsplit[2] in ['PKS_Docking_Nterm', 'PKS_Docking_Cterm']:
                    domaintype = domainsplit[2]
                else:
                    # We trim off the leading 'PKS_'
                    # Assume 'DH2' and 'DHt' are the same as 'DH' 
                    domaintype = domainsplit[2].split('_')[-1].replace('DHt', 'DH').replace('DH2', 'DH')
            # Special case of 'CAL' domain
            elif domainsplit[2] == 'CAL_domain':
                domaintype = 'CAL'
            else:
                domaintype = domainsplit[2]
        else:
            continue
    
        # DEBUG
#        print(domaintype)
    
        # These are the catlytic domains that we want to recognize
        if domaintype not in ['KS', 'AT', 'KR', 'DH', 'ER', 'ACP', 'Thioesterase', 
                              'cMT', 'oMT', 'CAL', 'PCP', 
                              'Heterocyclization', 'AMP-binding', 
                              'Condensation_DCL', 'Condensation_LCL',
                              'PKS_Docking_Nterm', 'PKS_Docking_Cterm']:
            # Break out of for loop and stop looking for additional catalytic domains if 
            # we encountered a domain that we don't recognize
            # we end up excluding any subunit that has a non-recognized catalytic domain
            # this is dealt with by checking subunits against those that are expected to be recognized
            # as determined by the MiBiG JSON file
            break    
        # Get the obundaries of the catalytic domain
        boundaries = [int(bound) for bound in domainsplit[3].replace('(', '').replace(')', '').replace('.', '').split('-')]
        
        # Here, we add each domain to a list, which will be converted to an OrderedDict
        # based on whether or not the domain is expected to have substrate specificity annotations
        if domaintype in ['KS', 'DH', 'ER', 'ACP', 'cMT', 'oMT', 'CAL', 'PCP',
                          'Heterocylization', 'AMP-binding', 
                          'Condensation_DCL', 'Condensation_LCL',
                          'PKS_Docking_Nterm', 'PKS_Docking_Cterm']:   # Recall that we trimmed leading 'PKS_'
            module_domains.append((domaintype, [{'start': boundaries[0], 'stop': boundaries[1]}]))
        # Include substrate and stereospecificity annotations for AT and KR domains respectively
        if domaintype in ['AT', 'KR']:   # Recall that we trimmed leading 'PKS_'
            notesdict = {}
            for note in entrysplit[1:]:
                item = note.split(': ')
                notesdict[item[0]] = item[1]
            module_domains.append((domaintype, [{'start': boundaries[0], 'stop': boundaries[1]}, notesdict]))
                
        # End of the module has been reached of the domain is 'ACP' or 'PCP
        if domaintype in ['ACP', 'PCP']:
            domains_present = [d[0] for d in module_domains]
            # Make sure every module has an AT, or else it isn't a valid module and we just ignore it
            # This means it will be excluded from the subunit, which makes sense since we can't 
            # really perform a polyketide chain extension without an AT
            if 'AT' in domains_present:            
                subunit[module_index] = OrderedDict(module_domains)
                old_module_domains = module_domains
                module_index += 1
            else:
                old_module_domains = []
            module_domains = []
        # These domains may come after the ACP or PCP, so if they are encountered, we add
        # them to previous module and keep going forward
        if domaintype in ['Thioesterase', 'PKS_Docking_Cterm', 'Condensation_LCL']:
            # Overwrite previous subunit, or else will have duplicate entries
            old_module_domains.append((domaintype, [{'start': boundaries[0], 'stop': boundaries[1]}]))
            subunit[module_index - 1] = OrderedDict(old_module_domains)
            module_domains = []
            
    return subunit

In [ ]:
def get_gene_data(record):
    '''Takes as input a record read in from an MiBiG GenBank file using SeqIO.read() and outputs PKS data 
       from that record. Data will be comprised of PKS subunits and standalone PKS genes.
    '''
    # Get list to hold information about all genes that are in the record
    gene_data = []
    
    # Only the "CDS" features are potentially genes
    # Here we get genes that aren't necessarily subunits
    for feature in record.features:
        # These are the features we are interested in
        if feature.type == 'CDS' and 'protein_id' in feature.qualifiers.keys() and 'gene' in feature.qualifiers.keys(): 
            # This gets the location of the feature
            location = feature.location
            # Potential information about gene
            if 'product' in feature.qualifiers.keys():
                description = feature.qualifiers['product'][0]
            gene_data.append([feature.qualifiers['protein_id'][0],
                              feature.qualifiers['gene'][0],
                             ])
            # Feature may not be a PKS module and therefore may not have have subunits 
            # (this will be overwritten if it does have subunits)
            subunit_modules = None
            # Information if gene is PKS subunit
            if 'sec_met' in feature.qualifiers.keys() and len(feature.qualifiers['sec_met']) > 3:
                if feature.qualifiers['sec_met'][3] in ['NRPS/PKS subtype: Type I Modular PKS', 
                                                        'NRPS/PKS subtype: PKS-like protein',
                                                        'NRPS/PKS subtype: PKS/NRPS-like protein',
                                                        'NRPS/PKS subtype: Hybrid PKS-NRPS']:                    
                    # DEBUG
#                    print(feature.qualifiers['gene'][0])
                    # This gets the subunit information                    
                    subunit_modules = process_subunit_modules(feature.qualifiers['sec_met'])
#                else:
#                    print(feature)
            
            # More general information
            gene_data[-1].extend([description, [location.start.position, location.end.position]])

            # Subunit information (if it doesn't have subunit information, assumed to be a standalone enzyme)
            if subunit_modules:
                gene_data[-1].append(subunit_modules)

            # General information about gene
            gene_data[-1].append(feature.qualifiers['translation'][0])

    return gene_data

In [ ]:
def check_json_module_validity(module_list):
    '''Function that makes sure module specified in JSON file is valid,
       that is to say, make sure that it contains KS, AT, and ACP or PCP.
       AT least as of February 24, 2017, the names of these domains appear
       only in the following forms in clusters that are annotated as Type I modular PKSs
       ['KS', 'AT', 'T']
       ['Ketosynthase', 'Acyltransferase', 'Thiolation (ACP/PCP)']
    '''
    at_check = len(set(['AT', 'Acyltransferase']).intersection(set(module_list)))
    acp_check = len(set(['ACP', 'PCP', 'T', 'Thiolation (ACP/PCP)']).intersection(set(module_list)))

    if at_check and acp_check:
        return True
    else:
        return False

def process_cluster(record, mibig_json):
    '''Takes in a record and corresponding MiBiG json file
       then returns pks.Cluster object representing the cluster.
    '''
    # Get information about the gene
    gene_data = get_gene_data(record)
    if len(gene_data) == 0:
        return

    # Initalize lists for subunits and standalones
    # We make two dictionaries because sometimes the subunit name in the MiBiG JSON files
    # is the gene name, e.g. eryA1, and sometimes it is the accession number, e.g. A0000000
    unordered_subunits = {}
    unordered_subunits_alt = {}
    standalones = []
     
    # Recall that each entry in gene_data is a list
    # [protein id, gene, product, [location start, location end], subunit dict (optional), translation]
    
    #####################
    # Basic information #
    #####################
    
    counter = 1
    for gene in gene_data:
        geneid = gene[0].strip()
        genename = gene[1].strip()
        genedesc = gene[2].strip()
        genestart = gene[3][0]
        genestop = gene[3][1]
        genetranslation = gene[-1].strip()

        # Just use length of gene_data to differentiate between standalones and subunits
        if len(gene) == 6:
            # We do this to take care of duplicated gene names, as is the case wity tylactone (BGC0000166)
            if genename in unordered_subunits_alt.keys():
                genename = genename + '_' + str(counter)
                counter += 1
 
            # Get subunit data from gene
            genesubunitdata = gene[-2]
            # Here we use the two dictionary options to save the unordered subunits
            # Sometimes MiBiG uses geneid and sometimes it uses genename to reference subunits
            unordered_subunits[geneid] = (genename, genedesc, genestart, genestop,
                                            genesubunitdata, genetranslation)
            unordered_subunits_alt[genename] = (geneid, genedesc, genestart, genestop,
                                                genesubunitdata, genetranslation)
        else:
            # Standalones lack subunit and orphan entries
            assert len(gene) == 5, gene
            standalones.append(pks.Standalone(geneid, genename, genedesc, 
                                              genestart, genestop, genetranslation))
        
    #########################################
    # JSON file has cyclization information #
    #########################################

    # Get ordered version of subunits from corresponding JSON file
    with open(mibig_json) as json_file:
        mibig_data = json.load(json_file)
    
    # Get PKS cyclization information
    # this will be either 'Cyclic' or 'Linear'
    try:
        lin_cycl_pk = mibig_data['general_params']['Polyketide']['lin_cycl_pk']
        if lin_cycl_pk == 'Cyclic':
            cyclize = 'Cyclic'
        elif lin_cycl_pk == 'Linear':
            cyclize = 'Linear'
        else:
            raise Exception("lin_cycl_pk expected to be 'Cyclic' or 'Linear'.")
    except KeyError:
        cyclize = 'Linear'
            
    #####################################
    # JSON file has subunit information #
    #####################################
        
    # Note that all gene data has now been processed, want to reprocess to get right ordering 
    # We strip out subunits that have invalid modules
    try:
        ordered_subunits = []
        for subunit in mibig_data['general_params']['Polyketide']['mod_pks_genes']:
            subunit_name = re.sub(r'\s+', '', subunit['mod_pks_gene'])
            subunit_modules = subunit['pks_module']

            valid_subunit = True
            # This checks if the module is valid
            for module in subunit_modules:
                # Just for debugging
    #            print(module['pks_domains'])
                if not check_json_module_validity(module['pks_domains']):
                    valid_subunit = False
            if valid_subunit:
                ordered_subunits.extend(subunit_name.split(','))
            else:
                # Loop is broken once first invalid subunit is encountered
                break
        # If no valid subunits, then just return
        if len(ordered_subunits) == 0:
            print('\tNo valid subunits!')
            for subunit in mibig_data['general_params']['Polyketide']['mod_pks_genes']:
                subunit_name = re.sub(r'\s+', '', subunit['mod_pks_gene'])
                subunit_modules = subunit['pks_module']
                for module in subunit_modules:
                    print(module['pks_domains'])
            return
        # This makes sure the subunit accession naming is consistent
        # The purpose of these two 'if' statements is because there may be cases in the MiBiG JSON file
        # where the name of the gene is for example, 'eryA1, A000000' and we want to keep consistant naming
        if len(ordered_subunits[0]) >= 8:
            ordered_subunits = [entry for entry in ordered_subunits if len(entry) >= 8]
        if len(ordered_subunits) > 1:
            if len(ordered_subunits[1]) >= 8:
                ordered_subunits = [entry for entry in ordered_subunits if len(entry) >= 8]
        # This is because sometimes the accession number under which the gene is recorded sometimes
        # has a version number, and sometimes does not
        if len(ordered_subunits[0].split('.')) == 1 and len(ordered_subunits[0]) == 8:
            ordered_subunits = [entry + '.1' for entry in ordered_subunits]

        # Check if subunit is in either dictionary
        for isubunit,subunit in enumerate(ordered_subunits):
            if subunit not in set(list(unordered_subunits.keys()) + list(unordered_subunits_alt.keys())):
                print('Missing subunit: "%s"' %(subunit))
                for gene in mibig_data['general_params']['Polyketide']['mod_pks_genes']:
                    if gene['mod_pks_gene'] == subunit:
                        module = gene['pks_module']
                        for entry in module:
                            print(entry['pks_domains'])
                print(unordered_subunits.keys())
                return
    #    print([gene_ref for gene_ref in mibig_data['general_params']['Polyketide']['mod_pks_genes']])

        # Determine whether to use standard or alternative dict
        if len(ordered_subunits[0]) >= 8:
            alt = False
        else:
            alt = True
    
    # Just use unordered gene order if the gene ordering is not already in the JSON file 
    except Exception:
        ordered_subunits = list(unordered_subunits_alt.keys())
        ordered_subunits.sort()
        alt = True

    ####################################
    # This does the subunit reordering #
    ####################################
    # Initialize final list of subunits
    subunits = []
    
    modules_seen = 0
    for subunit in ordered_subunits:
        # subunit data has form (id, description, start, stop, module dict, sequence)
        if not alt:
            subunitdata = unordered_subunits[subunit]
        else:
            subunitdata = unordered_subunits_alt[subunit]
        # Initialize list to hold processed modules
        # this is a list of module objects that we will pass to pks.Subunit()
        modules = []
        # This is the modules for the subunit
        moduledata = subunitdata[-2]
        
        # We do this so we can lump in the loading didomain and TE on the first and last modules respectively
        modulekeys = list(moduledata.keys())
        imodule = 0
        while imodule < len(modulekeys):
            # Get info
            keys = list(moduledata[modulekeys[imodule]].keys())
            values = moduledata[modulekeys[imodule]].values()
            # Process info according to loading or not
            if modules_seen == 0:
                loading = True                
                # Don't name KSQ and ATL separate after all
#                moduledict =  OrderedDict([(k.replace('KS', 'KS').replace('AT', 'AT'), v) \
#                                          if k in ['KS','AT'] \
#                                          else (k,v) \
#                                          for k,v in zip(keys,values)])
                moduledict =  OrderedDict([(k,v) for k,v in zip(keys,values)])
            else: 
                loading = False
                moduledict = OrderedDict([(k,v) for k,v in zip(keys,values)])
            # Determine whether module is terminal or not
            if 'Thioesterase' in list(moduledata[modulekeys[imodule]].keys()):
                terminal = cyclize
            else:
                terminal = None
            imodule += 1
            modules_seen += 1
            try:
                # This is to make sure we don't add subunits with invalid modules
                # The check for errors here is to compare agains the predicted chemcial structure
                domains_present = moduledict.keys()
                if 'ACP' in domains_present or 'PCP' in domains_present:
                    if 'AT' in domains_present or 'ATL' in domains_present:
                        modules.append(pks.Module(moduledict, loading=loading, terminal=terminal))
            except AssertionError as e:
                print(moduledict)
                print(type(e).__name__, e.args, subunit + ' ' + subunitdata[1])
                raise Exception(type(e).__name__, e.args, subunit + ' ' + subunitdata[1])
                break
        # Add subunit to list
        if len(modules) > 0:
            if not alt:
                subunits.append(pks.Subunit(subunit, subunitdata[0], subunitdata[1],
                                            subunitdata[2], subunitdata[3], subunitdata[-1],
                                            modules))
            else:
                subunits.append(pks.Subunit(subunitdata[0], subunit, subunitdata[1],
                                            subunitdata[2], subunitdata[3], subunitdata[-1],
                                            modules))
        else:
            return

        # We take the last Subunit object and change the TE contained in the subunit to cyclize if necessary
        if cyclize:
            final_module = subunits[-1].modules[-1]
            if final_module.terminal == True:
                assert 'Thioesterase' in list(final_module.domains.keys()), \
                    "Terminal module lacks 'Thioesterase' in domains dictionary."
                TE = final_module.operations[-1]
                TE.cyclize = True
        
    return (subunits, standalones)

In [ ]:
# Testing specific entries
entry = ('BGC0000166', 'U78289')  # tylactone
#entry = ('BGC0000033', 'AF497482') # calicheamicin

with open(os.path.join(file_path, entry[0] + '.json')) as json_file:
    mibig_data = json.load(json_file)
#pprint(test_data.keys())
#pprint(test_data['general_params'].keys())
#pprint(test_data['general_params'])
#pprint([gene_ref['mod_pks_gene'] for gene_ref in test_data['general_params']['Polyketide']['mod_pks_genes']])
record = SeqIO.read(os.path.join(antismash_file_path, entry[0] + '.embl'), "embl")

# antismash_data = get_gene_data(record)
# for subunit in antismash_data:
#     if len(subunit) == 6:
#         print(subunit)

subunits, standalones = process_cluster(record, os.path.join(file_path, entry[0] + '.json'))
print(subunits)

# Process all clusters

In [ ]:
import imp
helper = imp.load_source('helper', './pks/helper.py')
domain = imp.load_source('domain', './pks/domain.py')
pks = imp.load_source('pks', './pks/pks.py')

In [ ]:
entry = ('BGC0000166', 'U78289')
record = SeqIO.read(os.path.join(antismash_file_path, entry[0] + '.embl'), "embl") 
#print(record.__dict__)
print(record.annotations['comment'].split()[-1].strip().strip('.'))

In [ ]:
# Initialize dictionary to hold clusters and counter for valid clusters
cluster_dict = {}
valid = 0

# Iterate over list of type I modular PKSs
for i in range(len(t1pks)):
    print('%d: %s' %(i, t1pks[i]))
    entry = t1pks[i]

     # This prints the accession number and product compound of the cluster
    with open(os.path.join(file_path, entry[0] + '.json')) as json_file:
        mibig_data = json.load(json_file)
        pprint([compound['compound'] for compound in mibig_data['general_params']['compounds']])

    # Read in cluster data
    record = SeqIO.read(os.path.join(antismash_file_path, entry[0] + '.embl'), "embl")    

    # Basic information
    cluster_id = record.id
    cluster_name = record.annotations['comment'].split()[-1].strip().strip('.')
    cluster_description = record.description
    cluster_sequence = record.seq
    
    # Subunit information
    try:
        subunits, standalones = process_cluster(record, os.path.join(file_path, entry[0] + '.json'))
        cluster = pks.Cluster(cluster_id, cluster_name, cluster_description, cluster_sequence,
                              subunits, standalones)
        cluster_dict[cluster_id] = cluster
        valid += 1
    # If we can't process the cluster, then we show the exception
    # We just use this for troubleshooting
    except Exception as e:
        print(Exception(type(e).__name__, e.args))

# Print number of valid clusters
print(valid)

In [ ]:
# Use this to filter out clusters that don't seem to have any subunits
npks_clusters = 0
pruned_clusters = {}

for key,value in cluster_dict.items():
    cluster = value
    subunits = cluster.subunits
    if subunits and len(list(subunits.keys())) > 0:
        print(cluster.description)
#        print('\t' + str(subunits))
        pruned_clusters[key.split('.')[0]] = value
        npks_clusters += 1

print(npks_clusters)

In [ ]:
starter_units = []
extender_units = []
nmodules = 0
for key,value in pruned_clusters.items():
#    print key
    cluster = value
    print(cluster.description)
    subunits = cluster.subunits
    if len(subunits) == 0:
        raise('Cluster has no subunits!')
    for key,value in subunits.items():
        subunit = value
        print('\t' + subunit.description)
        for module in subunit.modules:
            print('\t' + str(list(module.domains.keys())))
            print('\t' + str(module.operations))
            nmodules += 1
            if module.loading == True:
#                print '\t' + module.domains['ATL'][1]['Substrate specificity predictions']
                starter_units.append(module.domains['AT'][1]['Substrate specificity predictions'].split()[0])
            else:
#                print '\t' + module.domains['AT'][1]['Substrate specificity predictions']
                extender_units.append(module.domains['AT'][1]['Substrate specificity predictions'].split()[0])
print(nmodules)

In [ ]:
print(set(starter_units))
print(set(extender_units))

# Testing a single cluster

In [ ]:
from IPython.display import display
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
test_cluster_key = 'BGC0001053'
test_cluster = pruned_clusters[test_cluster_key]

In [ ]:
print(test_cluster_key)
print(test_cluster.description)
#print(list(test_cluster.subunits.values())[0].modules[0].operations[0].starter_name)
subunits = list(test_cluster.subunits.values())
chain = None
for subunit in subunits:
    print(subunit.name)
    print(subunit.id)
    for module in subunit.modules:
        print(str(list(module.domains.keys())))
        print(module.operations)
    chain = subunit.compute_product(chain)
    m_im = Draw.MolsToGridImage([chain], legends=[test_cluster.description], 
                                molsPerRow=1, subImgSize=(500,150), useSVG=True)
    display(m_im)

# Manually reorder clusters as needed

In [ ]:
pruned_clusters['BGC0000097'].print_domains()

In [ ]:
# BE-14106
pruned_clusters['BGC0000029'].update_subunit_order(['becB', 'becD', 'becE', 'becF', 'becG'])
pruned_clusters['BGC0000029'].toggle_cyclization()
# Cremimycin
#pruned_clusters['BGC0000042'].update_subunit_order(['cmiP7', 'cmiP8', 'cmiP1', 'cmiP2', 'cmiP3', 'cmiP4', 'cmiP5', 'cmiP6'])
# Meilingmycin
pruned_clusters['BGC0000093'].pop_subunit('pks1')
pruned_clusters['BGC0000093'].pop_subunit('pks2')
# ML-449
pruned_clusters['BGC0000097'].update_subunit_order(['mlaB', 'mlaD', 'mlaE', 'mlaF', 'mlaG'])
pruned_clusters['BGC0000097'].toggle_cyclization()
# Nystatin
pruned_clusters['BGC0000115'].update_subunit_order(['nysA', 'nysB', 'nysC', 'nysI', 'nysJ', 'nysK'])
# Tiacumicin
pruned_clusters['BGC0000165'].pop_subunit('tiaB')
# Simocyclinone
pruned_clusters['BGC0001072'].update_subunit_order(['simC1A', 'simC1B', 'simC1C'])
# Brasilinolide
pruned_clusters['BGC0001381'].update_subunit_order(['nbrI', 'nbrJ', 'nbrK', 'nbrL', 'nbrG', 'nbrH'])

# Testing all clusters

In [ ]:
pickle.dump(pruned_clusters, open('pruned_clusters_mibig.p', 'wb'))

In [ ]:
valid = 0
nclusters = len(pruned_clusters)

# Save invalid clusters so we can take a closer look
pruned_clusters_invalid = {}

for icluster,test_cluster_key in enumerate(list(pruned_clusters.keys())):
    try:
        test_cluster = pruned_clusters[test_cluster_key]
        print('(%d/%d): %s' %(icluster, nclusters, test_cluster_key))
        print(test_cluster.description)
        print('\tStarter from antiSMASH: %s' %(list(test_cluster.subunits.values())[0].modules[0].operations[0].starter_name))
        with open(os.path.join(file_path, test_cluster_key + '.json')) as json_file:            
            mibig_data = json.load(json_file)
            try:
                mibig_starter = mibig_data['general_params']['Polyketide']['starter_unit']
            except KeyError:
                mibig_starter = 'Not provided'
        print('\tStarter from MiBiG: %s' %(mibig_starter))        
        subunits = list(test_cluster.subunits.values())
        for subunit in subunits:
            print(subunit.name)
            for module in subunit.modules:
                print('\t' + str(list(module.domains.keys())))
        m = test_cluster.compute_product(None)
        m_im = Draw.MolsToGridImage([m], legends=[test_cluster.description], 
                                    molsPerRow=1, subImgSize=(500,150), useSVG=True)
        display(m_im)
        valid += 1
    except Exception as e:
        print('PREDICTION WARNING: %s' %(e))
        
        # Add to dictionary of invalid clusters
        pruned_clusters_invalid[test_cluster_key] = test_cluster

In [ ]:
print(valid)

In [ ]:
# Save invalid clusters so we can take a closer look
pruned_clusters_invalid = {}

for icluster,test_cluster_key in enumerate(list(pruned_clusters_invalid.keys())):
    try:
        test_cluster = pruned_clusters[test_cluster_key]
        print('(%d/%d): %s' %(icluster, nclusters, test_cluster_key))
        print('\tStarter from antiSMASH: %s' %(list(test_cluster.subunits.values())[0].modules[0].operations[0].starter_name))
        with open(os.path.join(file_path, test_cluster_key + '.json')) as json_file:            
            mibig_data = json.load(json_file)
            try:
                mibig_starter = mibig_data['general_params']['Polyketide']['starter_unit']
            except KeyError:
                mibig_starter = 'Not provided'
        print('\tStarter from MiBiG: %s' %(mibig_starter))        
        subunits = list(test_cluster.subunits.values())
        for subunit in subunits:
            print(subunit.name)
            for module in subunit.modules:
                print('\t' + str(list(module.domains.keys())))
        m = test_cluster.compute_product(None)
        m_im = Draw.MolsToGridImage([m], legends=[test_cluster.description], 
                                    molsPerRow=1, subImgSize=(500,150), useSVG=True)
        display(m_im)
        valid += 1
    except Exception as e:
        print('WARNING: %s' %(e))

# Printing subunit amino acid sequences

In [ ]:
# Make sure all names are distinct
subunit_names = []
for icluster,test_cluster_key in enumerate(list(pruned_clusters.keys())):
    test_cluster = pruned_clusters[test_cluster_key]
    subunits = list(test_cluster.subunits.values())
    for subunit in subunits:
        subunit_names.append(test_cluster_key + subunit.name)
print(len(subunit_names))
print(len(set(subunit_names)))

In [ ]:
def write_fasta_for_cluster(cluster):
    with open('./subunit_aa_sequences/' + str(cluster.name) + '.fasta', 'w') as f:
        subunits = list(test_cluster.subunits.values())
        for subunit in subunits:
            sequence = subunit.sequence
            # Write subunit to fasta file
            f.write('>' + cluster.name + ',' + subunit.name + '\n')
            # In fasta format each line has only 80 characters
            index = 0
            written = 0
            size = 80
            while index < len(sequence) - 1:
                try:
                    segment = sequence[written*size:(written+1)*size]
                except IndexError:
                    segment = sequence[written*size:len(sequence - 1)]
                f.write(segment + '\n')
                index += size
                written += 1
        
nclusters = len(pruned_clusters)
nsubunits = 0
for icluster,test_cluster_key in enumerate(list(pruned_clusters.keys())):
    test_cluster = pruned_clusters[test_cluster_key]
    print('(%d/%d): %s' %(icluster, nclusters, test_cluster_key))
    write_fasta_for_cluster(test_cluster)
    nsubunits += len(test_cluster.subunits)
print('Wrote out %d subunits from %d clusters.' %(nsubunits, nclusters))